In [11]:
# Required Libraries
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

In [19]:
# Load datasets
sector_inter = pd.read_csv('sector_22132100_interpolated.csv', parse_dates=['Unnamed: 0']).set_index('Unnamed: 0')
future_data = pd.read_csv('future_data2.csv', parse_dates=['RowKey']).set_index('RowKey')

In [31]:
sector_inter.index.names = ['RowKey']

In [21]:

# Splitting the data
train, valid = train_test_split(sector_inter, test_size=0.2, shuffle=False)
test = future_data

In [22]:

# Drop INF_Value for XGBoost training and validation
X_train = train.drop('INF_Value', axis=1)
y_train = train['INF_Value']
X_valid = valid.drop('INF_Value', axis=1)
y_valid = valid['INF_Value']
X_test = test.drop('INF_Value', axis=1)

In [16]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import itertools

# Define the range of orders and seasonal orders to search through
p_values = [0, 1, 2]  # AutoRegressive order (p)
d_values = [0, 1]     # Differencing order (d)
q_values = [0, 1, 2]  # Moving Average order (q)
seasonal_periods = [7, 30, 13, 12]  # Different seasonal periods to try

best_aic = float("inf")
best_order = (0, 0, 0)
best_seasonal_order = (0, 0, 0, 0)

# Iterate through all possible combinations
for p in p_values:
    for d in d_values:
        for q in q_values:
            for seasonal_period in seasonal_periods:
                order = (p, d, q)
                seasonal_order = (p, d, q, seasonal_period)
                
                # Fit the SARIMA model with the current order and seasonal order
                sarima_model = SARIMAX(y_train, order=order, seasonal_order=seasonal_order)
                sarima_result = sarima_model.fit(disp=False)
                
                # Calculate AIC (Akaike Information Criterion)
                current_aic = sarima_result.aic
                
                # Update the best order if the current AIC is lower
                if current_aic < best_aic:
                    best_aic = current_aic
                    best_order = order
                    best_seasonal_order = seasonal_order

# Print the best SARIMA order and seasonal order
print("Best SARIMA Order:", best_order)
print("Best Seasonal Order:", best_seasonal_order)

C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCac

C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Pytho

Best SARIMA Order: (0, 1, 1)
Best Seasonal Order: (0, 1, 1, 30)


In [23]:
# Fit the SARIMA model with the best order and seasonal order
best_sarima_model = SARIMAX(y_train, order=best_order, seasonal_order=best_seasonal_order)
best_sarima_result = best_sarima_model.fit(disp=False)

# SARIMA predictions with the best orders
sarima_predictions = best_sarima_result.predict(start=y_train.index[-1], end=y_valid.index[-1], dynamic=True)
future_sarima_predictions = best_sarima_result.predict(start=y_valid.index[-1], end=test.index[-1], dynamic=True)
# Fit the SARIMA model with the best order and seasonal order
best_sarima_model = SARIMAX(y_train, order=best_order, seasonal_order=best_seasonal_order)
best_sarima_result = best_sarima_model.fit(disp=False)

# SARIMA predictions with the best orders
sarima_predictions = best_sarima_result.predict(start=y_train.index[-1], end=y_valid.index[-1], dynamic=True)
future_sarima_predictions = best_sarima_result.predict(start=y_valid.index[-1], end=test.index[-1], dynamic=True)
sarima_predictions = abs(sarima_predictions)
future_sarima_predictions = abs(future_sarima_predictions)

C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\statespace\kalman_filter.py:2473: ValueWarning: Dynamic prediction specified to begin during out-of-sample forecasting period, and so has no effect.
  warn('Dynamic prediction specified to begin during'
C:\Users\lalit\AppData\Local\Packages\Py

In [18]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

# Define the parameter grid to search
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6],
    'min_child_weight': [1, 2, 3],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'n_estimators': [50, 100, 200]
}

# Create the XGBoost model
xgb_model = XGBRegressor()

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the grid search to your training data
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)



Best Hyperparameters: {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 100, 'subsample': 0.7}


ValueError: feature_names mismatch: ['precip_lag1', 'precip_lag2', 'precip_lag3', 'precip_lag4', 'precip_lag5', 'precip_lag6', 'precip_lag7', 'precip_rolling_sum', 'precip_rolling_mean', 'precip_rolling_max', 'precip_rolling_min', 'no_rain', 'light_rain', 'heavy_rain', 'days_since_last_rain', 'accumulated_precip_30d', 'precip_change', 'temp_lag1', 'temp_lag2', 'temp_lag3', 'humidity_lag1', 'humidity_lag2', 'humidity_lag3', 'temp_rolling_mean', 'temp_rolling_std', 'humidity_rolling_mean', 'humidity_rolling_std', 'INF_Value_Interpolated'] ['precip_lag1', 'precip_lag2', 'precip_lag3', 'precip_lag4', 'precip_lag5', 'precip_lag6', 'precip_lag7', 'precip_rolling_sum', 'precip_rolling_mean', 'precip_rolling_max', 'precip_rolling_min', 'no_rain', 'light_rain', 'heavy_rain', 'days_since_last_rain', 'accumulated_precip_30d', 'precip_change', 'temp_lag1', 'temp_lag2', 'temp_lag3', 'humidity_lag1', 'humidity_lag2', 'humidity_lag3', 'temp_rolling_mean', 'temp_rolling_std', 'humidity_rolling_mean', 

In [24]:
# Train a new XGBoost model with the best hyperparameters
best_xgb_model = XGBRegressor(**best_params)
best_xgb_model.fit(X_train, y_train)

# XGBoost predictions with the best hyperparameters
xgb_predictions = best_xgb_model.predict(X_valid)
future_xgb_predictions = best_xgb_model.predict(X_test)
# XGBoost predictions with the best hyperparameters
xgb_predictions = best_xgb_model.predict(X_valid)
future_xgb_predictions = best_xgb_model.predict(X_test)
# Convert xgb_predictions to a pandas Series with a datetime index
xgb_predictions_series = pd.Series(xgb_predictions, index=y_valid.index)
future_xgb_predictions_series = pd.Series(future_xgb_predictions, index=test.index)

In [33]:

future_ensemble_predictions = 0.65 * future_sarima_predictions + 0.35 * future_xgb_predictions_series

# Ensemble Predictions
ensemble_predictions = 0.65 * sarima_predictions + 0.35 * xgb_predictions_series

In [34]:
# Create DataFrame for Test and Future Predictions
predictions_df = pd.DataFrame({
    'Actual': y_valid,
    'SARIMA_Predictions': sarima_predictions,
    'XGBoost_Predictions': xgb_predictions_series,
    'Ensemble_Predictions': ensemble_predictions
}, index=y_valid.index)

future_predictions_df = pd.DataFrame({
    'SARIMA_Future_Predictions': future_sarima_predictions,
    'XGBoost_Future_Predictions': future_xgb_predictions_series,
    'Ensemble_Future_Predictions': future_ensemble_predictions
}, index=test.index)

In [35]:


import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Actual values
actual_values = predictions_df['Actual']

# SARIMA predictions
sarima_predictions = predictions_df['SARIMA_Predictions']

# XGBoost predictions
xgboost_predictions = predictions_df['XGBoost_Predictions']

# Ensemble predictions
ensemble_predictions = predictions_df['Ensemble_Predictions']

# Calculate MAPE for each method
def mape(y_true, y_pred):
    return 100 * np.mean(np.abs((y_true - y_pred) / y_true))

mape_sarima = mape(actual_values, sarima_predictions)
mape_xgboost = mape(actual_values, xgboost_predictions)
mape_ensemble = mape(actual_values, ensemble_predictions)

# Calculate RMSE for each method
rmse_sarima = np.sqrt(mean_squared_error(actual_values, sarima_predictions))
rmse_xgboost = np.sqrt(mean_squared_error(actual_values, xgboost_predictions))
rmse_ensemble = np.sqrt(mean_squared_error(actual_values, ensemble_predictions))

# Calculate MAE for each method
mae_sarima = mean_absolute_error(actual_values, sarima_predictions)
mae_xgboost = mean_absolute_error(actual_values, xgboost_predictions)
mae_ensemble = mean_absolute_error(actual_values, ensemble_predictions)

# Display the results
print("SARIMA MAPE:", mape_sarima)
print("XGBoost MAPE:", mape_xgboost)
print("Ensemble MAPE:", mape_ensemble)

print("SARIMA RMSE:", rmse_sarima)
print("XGBoost RMSE:", rmse_xgboost)
print("Ensemble RMSE:", rmse_ensemble)

print("SARIMA MAE:", mae_sarima)
print("XGBoost MAE:", mae_xgboost)
print("Ensemble MAE:", mae_ensemble)
future_predictions_df
predictions_df

SARIMA MAPE: 17.847530587464536
XGBoost MAPE: 26.48721045341308
Ensemble MAPE: 7.724732396133066
SARIMA RMSE: 43.52786678679499
XGBoost RMSE: 71.4159897586805
Ensemble RMSE: 19.987120778412113
SARIMA MAE: 32.13126448921289
XGBoost MAE: 48.43940026965142
Ensemble MAE: 15.251071172038616


,Actual,SARIMA_Predictions,XGBoost_Predictions,Ensemble_Predictions
RowKey,,,,
2022-09-04,233.77,223.116175,210.803040,218.806573
2022-09-05,221.57,221.015506,212.113663,217.899863
2022-09-06,233.54,224.406932,205.958359,217.949932
2022-09-07,223.30,230.681403,215.890335,225.504527
2022-09-08,237.87,234.754287,208.543335,225.580452
...,...,...,...,...
2022-12-08,163.56,242.422973,89.967926,189.063706
2022-12-09,168.52,241.737591,69.277405,181.376526
2022-12-10,167.96,248.838061,74.753494,187.908463


In [32]:
#mean of actual values
mean = predictions_df['Actual'].mean()
mean

208.8816

In [36]:
future_predictions_df

,SARIMA_Future_Predictions,XGBoost_Future_Predictions,Ensemble_Future_Predictions
RowKey,,,
2022-12-13,259.261871,36.461052,181.281584
2022-12-14,262.045028,49.181053,187.542635
2022-12-15,265.568230,-1.979118,171.926658
2022-12-16,253.518889,33.263248,176.429415
2022-12-17,262.145045,77.237289,197.427330
2022-12-18,259.408441,118.558250,210.110875
2022-12-19,247.547332,116.032509,201.517143
